In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import roc_auc_score

from importlib import reload
import matplotlib.pyplot as plt

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import model_params as params
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers

from tensorboardX import SummaryWriter
%load_ext line_profiler

In [2]:
device = torch.device('cuda:5')

dataset = 'FMNIST'

model_params = params.params_dict[dataset](augm_flag=True, batch_size=128)

In [6]:
file = 'base_FMNIST_lr0.1_augm_flagTrue_train_typeACET_steps40.pth'
model = torch.load('Checkpoints/' + file).to(device)

train_loader = model_params.train_loader

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `

In [4]:
def train_ACET(model, device, train_loader, noise_loader, optimizer, epoch, lam=1., steps=40, epsilon=0.3, verbose=100):
    criterion = nn.NLLLoss()
    model.train()
    
    train_loss = 0
    correct = 0
    
    enum = enumerate(train_loader)
    for batch_idx, (data, target) in enum:
        data, target = data.to(device), target.to(device)
        noise = torch.rand_like(data)

        output = model(data)
        
        model.eval()
        adv_noise = (adv.gen_adv_noise(model, device, noise, epsilon=epsilon, steps=steps) 
                     + 1e-4*torch.rand_like(noise))
        output_adv = model(noise)
        model.train()
        
        loss = criterion(output, target) - output_adv.mean()
        #loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)
    return train_loss/len(train_loader.dataset), correct/len(train_loader.dataset)

In [5]:
decay = 5e-4
lr = 0.1
param_groups = [{'params':model.parameters(),'lr':lr, 'weight_decay':decay}]

optimizer = optim.SGD(param_groups, momentum=0.9)

In [8]:
train_ACET(model, device, train_loader, train_loader, optimizer, 0)

tensor(2.9302, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.9408, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7733, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8589, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7050, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8155, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8148, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.9159, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8498, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7881, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8100, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7881, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.6120, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7171, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.6264, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7185, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7689, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7672, device='cuda:5', grad_fn=<SubBack

tensor(2.8608, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7080, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8500, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8358, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8323, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8664, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8484, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7908, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8204, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.7914, device='cuda:5', grad_fn=<SubBackward0>)
tensor(2.8198, device='cuda:5', grad_fn=<SubBackward0>)


KeyboardInterrupt: 